In [ ]:
# @title 🚀 Run AI Interview System
import os
import subprocess
import threading
import time

# --- 1. INSTALL DEPENDENCIES ---
print("Installing libraries... (This takes ~1 min)")
!pip install -q flask langchain-community langchain-core pypdf pyngrok
!apt-get install zstd -y # Install zstd for Ollama

# --- 2. INSTALL & START OLLAMA (AI BACKEND) ---
print("Installing Ollama AI...")
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama in the background
def start_ollama():
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

# Give Ollama time to start
time.sleep(10)

# Pull the Model (Using 'gemma:2b' as it is fast and lightweight for Colab)
print("Downloading AI Model (Gemma:2b)... this may take 2-3 mins")
!ollama pull gemma:2b

# --- 3. FLASK APP CODE ---
from flask import Flask, render_template_string, request, jsonify
from pypdf import PdfReader # Corrected import from PyPDF2 to pypdf
from langchain_community.llms import Ollama
from pyngrok import ngrok

app = Flask(__name__)

# Initialize AI
llm = Ollama(model="gemma:2b")

# HTML Template (Embedded here so you don't need external files)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Interview Prep</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        body { background-color: #f4f7f6; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; }
        .chat-box { height: 400px; overflow-y: auto; border: 1px solid #ddd; background: white; padding: 20px; border-radius: 10px; }
        .message { margin-bottom: 15px; padding: 12px; border-radius: 8px; max-width: 80%; }
        .bot { background-color: #e9ecef; color: #333; float: left; clear: both; }
        .user { background-color: #0d6efd; color: white; float: right; clear: both; text-align: right; }
        .listening { animation: pulse 1.5s infinite; color: red; }
        @keyframes pulse { 0% { transform: scale(1); } 50% { transform: scale(1.1); } 100% { transform: scale(1); } }
    </style>
</head>
<body>
<div class="container mt-5">
    <h2 class="text-center mb-4"><i class="fas fa-robot"></i> AI-Powered Interview Coach</h2>

    <div class="card p-4 mb-4 shadow-sm">
        <h5>1. Setup Interview Profile</h5>
        <div class="row g-3">
            <div class="col-md-6"><input type="file" id="resumeFile" class="form-control" accept=".pdf"></div>
            <div class="col-md-4"><input type="text" id="jobRole" class="form-control" placeholder="Target Role (e.g. Data Scientist)"></div>
            <div class="col-md-2"><button onclick="uploadResume()" class="btn btn-success w-100">Start</button></div>
        </div>
    </div>

    <div class="card p-4 shadow-sm" id="interviewSection" style="display:none;">
        <div class="d-flex justify-content-between align-items-center mb-3">
            <h5>2. Mock Interview Session</h5>
            <span id="status" class="text-muted small">Ready</span>
        </div>
        <div id="chatBox" class="chat-box mb-3 clearfix"></div>
        <div class="input-group">
            <button onclick="startListening()" id="micBtn" class="btn btn-outline-danger"><i class="fas fa-microphone"></i></button>
            <textarea id="userAnswer" class="form-control" placeholder="Type or click mic to speak..." rows="2"></textarea>
            <button onclick="submitAnswer()" class="btn btn-primary">Submit</button>
            <button onclick="getQuestion()" class="btn btn-secondary">Next Question</button>
        </div>
    </div>
</div>

<script>
    let currentQuestion = "";
    const synth = window.speechSynthesis;
    const SpeechRecognition = window.SpeechRecognition || window.webkitSpeechRecognition;
    const recognition = new SpeechRecognition();
    recognition.lang = 'en-US';

    recognition.onresult = (event) => {
        document.getElementById('userAnswer').value = event.results[0][0].transcript;
        document.getElementById('status').innerText = "Voice captured!";
        document.getElementById('micBtn').classList.remove('listening');
    };
    recognition.onspeechend = () => { recognition.stop(); document.getElementById('micBtn').classList.remove('listening'); };

    function startListening() {
        document.getElementById('status').innerText = "Listening...";
        document.getElementById('micBtn').classList.add('listening');
        recognition.start();
    }

    function speakText(text) {
        if (text !== '') {
            const utterThis = new SpeechSynthesisUtterance(text);
            synth.speak(utterThis);
        }
    }

    async function uploadResume() {
        const fileInput = document.getElementById('resumeFile');
        const formData = new FormData();
        formData.append('resume', fileInput.files[0]);
        const response = await fetch('/upload_resume', { method: 'POST', body: formData });
        if(response.ok) {
            document.getElementById('interviewSection').style.display = 'block';
            appendMessage("Hello! I have analyzed your resume. Ready to start.", "bot");
            speakText("Hello! I have analyzed your resume. Ready to start.");
        }
    }

    async function getQuestion() {
        const role = document.getElementById('jobRole').value;
        appendMessage("Generating question...", "bot");
        const response = await fetch('/generate_question', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ role: role })
        });
        const data = await response.json();
        currentQuestion = data.question;
        document.getElementById('chatBox').lastChild.remove(); // Remove 'generating'
        appendMessage(currentQuestion, "bot");
        speakText(currentQuestion);
    }

    async function submitAnswer() {
        const answer = document.getElementById('userAnswer').value;
        appendMessage(answer, "user");
        document.getElementById('userAnswer').value = "";
        appendMessage("Analyzing...", "bot");
        const response = await fetch('/evaluate_answer', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ question: currentQuestion, answer: answer })
        });
        const data = await response.json();
        document.getElementById('chatBox').lastChild.remove(); // Remove 'analyzing'
        appendMessage("Feedback: " + data.feedback, "bot");
        speakText(data.feedback);
    }

    function appendMessage(text, sender) {
        const chatBox = document.getElementById('chatBox');
        const div = document.createElement('div');
        div.className = `message ${sender}`;
        div.innerText = text;
        chatBox.appendChild(div);
        chatBox.scrollTop = chatBox.scrollHeight;
    }
</script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
"""

# Store resume text in global variable (Simulated DB)
resume_storage = {}

def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/upload_resume', methods=['POST'])
def upload_resume():
    file = request.files['resume']
    text = extract_text_from_pdf(file)
    resume_storage['text'] = text
    return jsonify({"message": "Uploaded"})

@app.route('/generate_question', methods=['POST'])
def generate_question():
    role = request.json.get('role', 'Engineer')
    resume_text = resume_storage.get('text', '')[:2000]
    prompt = f"You are an interviewer. Based on resume: {resume_text} and role: {role}, ask ONE short technical question."
    response = llm.invoke(prompt)
    return jsonify({"question": response.strip()})

@app.route('/evaluate_answer', methods=['POST'])
def evaluate_answer():
    data = request.json
    prompt = f"Question: {data['question']}\nAnswer: {data['answer']}\nRate out of 10 and give 1 sentence feedback."
    response = llm.invoke(prompt)
    return jsonify({"feedback": response.strip()})

# --- 4. RUN FLASK ---
if __name__ == '__main__':
    # Use ngrok to expose the local Flask server to the internet
    # Note: If ngrok asks for a token, sign up at ngrok.com (free) and add:
    ngrok.set_auth_token("39OAdo5cxIH6QTdHSj3odrGxA1Y_2f9YkCehuVm1Nd3cmLy87") # Uncommented and ready for user to add token

    # Try public URL first
    try:
        public_url = ngrok.connect(5000).public_url
        print(f"🚀 YOUR APP IS LIVE HERE: {public_url}")
    except:
        print("Ngrok error. Make sure you have an auth token if required.")

    app.run(port=5000)

Installing libraries... (This takes ~1 min)


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Installing Ollama AI...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
#########################                                                 35.5%

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


############################################                              62.2%

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

🚀 YOUR APP IS LIVE HERE: https://sappy-unrecompensed-misti.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:27:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:27:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:27:33] "POST /upload_resume HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:27:57] "POST /evaluate_answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:28:21] "POST /evaluate_answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:28:28] "POST /generate_question HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:32:18] "POST /evaluate_answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:32:34] "POST /generate_question HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2026 13:33:25] "POST /generate_question HTTP/1.1" 200 -
INF